In [1]:
import tensorflow as tf
import random
import numpy as np
from common_functions import *

In [2]:
import datetime

In [3]:
print(datetime.datetime.strptime("2007-10-30", "%Y-%M-%d"))

2007-01-30 00:10:00


In [22]:
sess = tf.Session()
Input = tf.placeholder(tf.float32, [None, 3])
Labels = tf.placeholder(tf.float32, [None, 3])
Money_Line = tf.placeholder(tf.float32, [None, 3])

worth_0 = tf.layers.dense(inputs=Input, units = 256, activation=tf.nn.relu)
worth_1 = tf.layers.dense(inputs=worth_0, units = 256, activation=tf.nn.relu)
worth_2 = tf.layers.dense(inputs=worth_1, units= 256, activation=tf.nn.relu)

output = tf.layers.dense(inputs=worth_2, units=3, activation=tf.nn.softmax)
scale = tf.layers.dense(inputs=worth_2, units=1, activation = tf.nn.sigmoid)

profit = tf.reduce_sum(Labels* Money_Line*output*scale, axis=1) - tf.reduce_sum(scale, axis=1)

avg_profit = tf.reduce_mean(profit)
total_profit_ratio = tf.reduce_mean(profit) / tf.reduce_mean(scale)

loss = -(avg_profit + total_profit_ratio - 200*tf.nn.relu(0.05-avg_profit)) #average profit balances opportunity. Total profit ratio balances real return +avg_profit) #+


opt = tf.train.AdamOptimizer(3e-5).minimize(loss)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
#saver.restore()

In [5]:
with open("odds_example_soccer.csv", "r") as odds:
    o = odds.read()
    o = o.split("\n")[3:]
    
for i,x in enumerate(o):
    o[i] = o[i].split(",")



In [6]:
extracted = [extract_features_and_labels(i,18,19 ,23, 1) for i in o]
extracted = list(filter(lambda x: x != -1, extracted ))

random.shuffle(extracted)
cutoff = int(len(extracted)*0.9)

train = extracted[:cutoff]
#train = train+ [augment(x) for x in train]
test = extracted[cutoff:]

In [26]:
for i in range(100000):
    features, labels = pack_sample(train, 128)
    t_p, a_p, L, __ = sess.run([total_profit_ratio, avg_profit, loss, opt], feed_dict = {Input:features, Labels:labels, Money_Line: features[:,:3]})
    
    if i % 1000 == 0:
        print("Iteration: "+ str(i))
        print("Training loss: "+ str(L))
        print("Training average profit: "+ str(a_p))
        print("Training actual profit: "+ str(t_p))
        features, labels= pack_sample(test, -1) 
        t_p, scales, avg_prof, prof, out, L = sess.run([total_profit_ratio, scale, avg_profit, profit, output, loss], feed_dict = {Input:features, Labels:labels, Money_Line: features[:,:3]})   
        
        print("Testing loss: "+ str(L))
        print("Testing average profit: "+ str(avg_prof))
        print("Testing actual profit: "+ str(t_p))
  #      print("Scales: "+ str(scales[:100]))
  #      print("Testing accuracy: "+ str(a))
        print("Outputs: "+ str(out[:5]))
        print("Inputs: "+ str(features[:5]))
        print("Labels: "+ str(labels[:5]))
        print("Profits"+ str(prof[:5]))
        
        losses = []
        gains = []
        for x in prof:
            if x < 0 and abs(x) > 0.5:
                losses.append(x)
            elif x > 0 and abs(x) > 0.5:
                gains.append(x)
        try:
            print(sum(losses)/len(losses),sum(gains)/len(gains))
        except ZeroDivisionError:
            pass
        print(len(losses), len(gains))
        print(len(losses+gains), len(prof))

Iteration: 0
Training loss: -0.9261723
Training average profit: 0.0725956
Training actual profit: 0.8535767
Testing loss: -1.3505534
Testing average profit: 0.14012657
Testing actual profit: 1.2104268
Outputs: [[3.8686568e-11 8.7258869e-15 1.0000000e+00]
 [3.0677565e-04 2.0446970e-04 9.9948871e-01]
 [1.5079253e-03 1.3667202e-02 9.8482484e-01]
 [6.7679188e-03 4.7431529e-01 5.1891685e-01]
 [1.3549243e-05 7.0110275e-07 9.9998569e-01]]
Inputs: [[ 1.164    7.3138  14.8992 ]
 [ 1.76961  3.30714  4.95104]
 [ 1.8019   3.73042  4.32456]
 [ 2.24301  3.34995  3.16546]
 [ 1.47186  3.9923   6.6378 ]]
Labels: [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Profits[-0.05696012 -0.02293797 -0.0028419  -0.00170173 -0.06514845]
-0.8783531220295491 2.169821838707459
780 1640
2420 19405
Iteration: 1000
Training loss: -0.99649024
Training average profit: 0.12512964
Training actual profit: 0.8713606
Testing loss: -1.305127
Testing average profit: 0.16137967
Testing actual profit: 1.1437473
Outp

KeyboardInterrupt: 

In [ ]:
saver.save(sess, "models_soccer/model.ckpt")